In [14]:
import asyncio
from uuid import uuid4

from openai import OpenAI
from agents import (
    Agent,
    Runner,
    trace,
    ItemHelpers,
    MessageOutputItem,
    HandoffOutputItem,
    ToolCallOutputItem,
    FileSearchTool,
    function_tool,
)
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX
from dotenv import load_dotenv
load_dotenv()

# create openAI client
client = OpenAI()

In [15]:
# Upload a file once, reuse everywhere
path = "wine.pdf"
upload = client.files.create(
    file=open(path, "rb"),
    purpose="assistants",
)

In [16]:
# Retrieve and print the file ID
FILE_ID = upload.id
FILE_ID

'file-JcGw3XrWSfqZNHmUjNaYob'

In [17]:
# Create the vector store with code
vector_store = client.vector_stores.create(
    name = "Vector-Store-X"
)

In [18]:
vector_store.id

'vs_69513bb3985081919bcf1462666a04fe'

In [19]:
# Insert the file in the vector store
vs_file = client.vector_stores.files.create(
    vector_store_id = 'vs_69513b38efbc8191bc3ca7d1095828a1',
    file_id=FILE_ID,
)

In [20]:
instructions = f"""{RECOMMENDED_PROMPT_PREFIX}
You are Bitte's AI Sommelier:
Your task is to provide the user with a wine recommendation for their dish.
Use FileSearchTool to pick up to 3 wines from the vector store.
If the interaction has nothing to do with the wine, do the handoff.
"""
vs_id = "vs_683f09129dcc81919305e1510f0c9da4"

In [21]:
# Define the escalate to boss function
@function_tool(
    name_override = "escalate_to_boss",
    description_override="Talk to the guest about wine. If the topic is not wine, escalate to the boss"
)

async def escalate_to_boss(request: str) -> str:
  print(f"Escalating to the boss {request}")
  return "The boss will be here soon"

In [22]:
# Build the AI Agent
sommelier = Agent(
    name = "Bitte AI Sommelier",
    model = "gpt-4.1-mini",
    instructions = instructions,
    tools = [
        FileSearchTool(vector_store_ids = [vs_id],
                       max_num_results = 3),
        escalate_to_boss
    ]
)

In [ ]:
# Build the Chat interactions
async def chat():
  # Store the conversation
  history = []

  # while loop for the conversation
  while True:
    user = input("User: ")
    if user.lower() in {"exit", "quit"}:
      break

    history.append({"role": "user", "content": user})

    # Apply tracing to the conversation
    with trace("Bitte Sommelier", group_id = uuid4().hex):
      run = await Runner.run(sommelier, history)

    stop_chat = False

    for itm in run.new_items:
      if isinstance(itm, MessageOutputItem):
        print("Sommelier", ItemHelpers.text_message_output(itm))

      elif isinstance(itm, ToolCallOutputItem):
        print("Boss was summoned")
        stop_chat = True
        break

    if stop_chat:
      break

    history = run.to_input_list()

# Run the chat
await chat()